In [1]:
#dependencies
import re
from bs4 import BeautifulSoup 
from splinter import Browser
import requests
import pandas as pd
from selenium import webdriver 
from selenium.common.exceptions import TimeoutException
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.service import Service as BraveService
from webdriver_manager.core.utils import ChromeType
import json
import time
from tqdm.notebook import tqdm 

<h4> Start cells here if partial FIPS already available in {state}_fips_scraped file

In [2]:
fips_check_scraped = pd.read_csv("../../../data/state_data/geo/geo_fips/23/IL_fips_scraped.csv") 
fips_check_scraped  = fips_check_scraped.loc[:, ~fips_check_scraped.columns.str.contains('^Unnamed')] 
fips_check_scraped.head() 

,LoanNumber,BorrowerState,full_add,Lat,Long,FIPS_z
0,1060027105,IL,"1751 Lake Cook Road Suite 600, DEERFIELD, IL",42.150930,-87.870512,1.703180e+14
1,1255837210,IL,"One South Wacker Drive, 36th Floor, CHICAGO, IL",41.878166,-87.636672,1.703184e+14
2,1314447103,IL,"870 Oak ST. 0.0, Lake in the Hills, IL",42.189417,-88.309248,1.711187e+14
3,2083627203,IL,"700 Commerce Drive Suite 140, Oak Brook, IL",41.851485,-87.937622,1.704384e+14
4,2142698210,IL,"One E. Oak Hill Drive, Suite 200 0.0, Westmont...",41.815009,-87.951864,1.704384e+14


In [3]:
#Latitude, Longitude, not included
fips_check_scraped_notna = fips_check_scraped[fips_check_scraped["Lat"].notna()] # check to make sure no nulls in coords
fips_check_scraped_notna.head() 

,LoanNumber,BorrowerState,full_add,Lat,Long,FIPS_z
0,1060027105,IL,"1751 Lake Cook Road Suite 600, DEERFIELD, IL",42.150930,-87.870512,1.703180e+14
1,1255837210,IL,"One South Wacker Drive, 36th Floor, CHICAGO, IL",41.878166,-87.636672,1.703184e+14
2,1314447103,IL,"870 Oak ST. 0.0, Lake in the Hills, IL",42.189417,-88.309248,1.711187e+14
3,2083627203,IL,"700 Commerce Drive Suite 140, Oak Brook, IL",41.851485,-87.937622,1.704384e+14
4,2142698210,IL,"One E. Oak Hill Drive, Suite 200 0.0, Westmont...",41.815009,-87.951864,1.704384e+14


In [4]:
#FIPS not included, of Lat, Long available
fips_check_scraped_null = fips_check_scraped_notna[fips_check_scraped_notna["FIPS_z"].isnull()] # check to make sure no nulls in coords
fips_check_scraped_null #28093

,LoanNumber,BorrowerState,full_add,Lat,Long,FIPS_z
465,9471527103,IL,"833 W WASHINGTON BLVD, CHICAGO, IL",41.882856,-87.648738,NaN
1320,4357417000,IL,"18960 County Hwy 5 W, EAST DUBUQUE, IL",42.508105,-90.565332,NaN
1968,6227707103,IL,"22069 US HIGHWAY 34, PRINCETON, IL",40.392967,-104.634462,NaN
1992,4428188509,IL,"22069 US Highway 34, Princeton, IL",40.392967,-104.634462,NaN
2078,3984847201,IL,"14 N 845 US-20, Hampshire, IL",44.370091,-108.038415,NaN
3141,2252087100,IL,"14 N 850 Route 20, HAMPSHIRE, IL",42.468637,-73.419886,NaN
3185,3313488304,IL,"43451 N US Highway 41, Zion, IL",35.242488,-85.690479,NaN
3547,6478417409,IL,"1501 W. MAIN ST PO Drawer E, FAIRFIELD, IL",38.033561,-78.497707,NaN
3619,6829277004,IL,"43451 US HIGHWAY 41, ZION, IL",47.046908,-88.518619,NaN
3801,6463967004,IL,"11600 Sterling Parkway, RICHMOND, IL",42.495414,-88.320057,NaN


<h4>FIPS Script

In [5]:
#if FIPS_z file 
lat = fips_check_scraped_null["Lat"].astype(float).astype(str).to_list()
long = fips_check_scraped_null["Long"].astype(float).astype(str).to_list()

In [6]:
#before running script, check for any NaN within Lat, Long fields. 
def sleep():
    time.sleep(1)
option = webdriver.ChromeOptions()
option.binary_location = "/Applications/Brave Browser.app/Contents/MacOS/Brave Browser"
driver = Browser('chrome', options=option, headless=False)    
url = 'https://geocoding.geo.census.gov/geocoder/geographies/coordinates?form'
driver.visit(url)

not_found = []
geo_id = []

i=0
for i in tqdm(range(len(lat))):
    #while i < len(lat):
        driver.find_by_id('geoCoordX').clear()
        driver.find_by_id('geoCoordY').clear()
        driver.find_by_id('geoCoordX').fill(long[i])
        driver.find_by_id('geoCoordY').fill(lat[i])
        driver.find_by_id('geoCoordsSubmit').click()
        #sleep()
        #html = driver.html
        soup = BeautifulSoup(driver.html, 'html.parser')


        results = soup.find_all('div', id='geoCoordsJsonOutput')
        for result in results:
            result_text = result.text

            result_digits = re.findall('\d+', result_text)
            startswith = [s for s in result_digits if s.startswith('17')] #IL
            fips_elements =[element for element in startswith if len(element)>=15]
            
            try:
                #print(f"{i},{lat[i]},{long[i]} is {fips_elements}")
                geo_id.append(fips_elements)
            except (KeyError, IndexError):
                #print(f"{i},{lat[i]},{long[i]} Not Found")
                not_found.append(i)
                geo_id.append(None)  
            except (TimeoutException):
                driver.navigate.refresh()
    
        #sleep()
        i=i+1

  0%|          | 0/48 [00:00<?, ?it/s]

coordinates wtih more than one return:
385,37.514070000000004,-77.35472 is ['514070000000004', '510872015041003'] 
1178,37.516454700000004,-77.8100575 is ['516454700000004', '511455001022014'] 
2660,37.519462299999994,-77.340384 is ['519462299999994', '510872014061012']
second option reference FIPS code, first option references LatCoord, update fields in DataFrame

https://iqss.github.io/dss-webscrape/filling-in-web-forms.html

In [7]:
geo_id_list =geo_id 

Length Check

In [8]:
print(len(geo_id_list))

48


Matching FIPS to DataFrame

In [9]:
fips_fill = fips_check_scraped_null

In [10]:
fips_fill["FIPS_z"] = geo_id_list
fips_fill

/var/folders/vy/qdp8xcfj4s3dd5c6d1xp_y3w0000gn/T/ipykernel_10153/597263667.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  fips_fill["FIPS_z"] = geo_id_list


,LoanNumber,BorrowerState,full_add,Lat,Long,FIPS_z
465,9471527103,IL,"833 W WASHINGTON BLVD, CHICAGO, IL",41.882856,-87.648738,[170318330001046]
1320,4357417000,IL,"18960 County Hwy 5 W, EAST DUBUQUE, IL",42.508105,-90.565332,[]
1968,6227707103,IL,"22069 US HIGHWAY 34, PRINCETON, IL",40.392967,-104.634462,[]
1992,4428188509,IL,"22069 US Highway 34, Princeton, IL",40.392967,-104.634462,[]
2078,3984847201,IL,"14 N 845 US-20, Hampshire, IL",44.370091,-108.038415,[]
3141,2252087100,IL,"14 N 850 Route 20, HAMPSHIRE, IL",42.468637,-73.419886,[]
3185,3313488304,IL,"43451 N US Highway 41, Zion, IL",35.242488,-85.690479,[]
3547,6478417409,IL,"1501 W. MAIN ST PO Drawer E, FAIRFIELD, IL",38.033561,-78.497707,[]
3619,6829277004,IL,"43451 US HIGHWAY 41, ZION, IL",47.046908,-88.518619,[]
3801,6463967004,IL,"11600 Sterling Parkway, RICHMOND, IL",42.495414,-88.320057,[]


Remove brackets

In [11]:
fips_fill['FIPS_z'] = fips_fill['FIPS_z'].str[0]
fips_fill

/var/folders/vy/qdp8xcfj4s3dd5c6d1xp_y3w0000gn/T/ipykernel_10153/1219885402.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  fips_fill['FIPS_z'] = fips_fill['FIPS_z'].str[0]


,LoanNumber,BorrowerState,full_add,Lat,Long,FIPS_z
465,9471527103,IL,"833 W WASHINGTON BLVD, CHICAGO, IL",41.882856,-87.648738,170318330001046
1320,4357417000,IL,"18960 County Hwy 5 W, EAST DUBUQUE, IL",42.508105,-90.565332,NaN
1968,6227707103,IL,"22069 US HIGHWAY 34, PRINCETON, IL",40.392967,-104.634462,NaN
1992,4428188509,IL,"22069 US Highway 34, Princeton, IL",40.392967,-104.634462,NaN
2078,3984847201,IL,"14 N 845 US-20, Hampshire, IL",44.370091,-108.038415,NaN
3141,2252087100,IL,"14 N 850 Route 20, HAMPSHIRE, IL",42.468637,-73.419886,NaN
3185,3313488304,IL,"43451 N US Highway 41, Zion, IL",35.242488,-85.690479,NaN
3547,6478417409,IL,"1501 W. MAIN ST PO Drawer E, FAIRFIELD, IL",38.033561,-78.497707,NaN
3619,6829277004,IL,"43451 US HIGHWAY 41, ZION, IL",47.046908,-88.518619,NaN
3801,6463967004,IL,"11600 Sterling Parkway, RICHMOND, IL",42.495414,-88.320057,NaN


In [12]:
fips_fill.loc[fips_fill["FIPS_z"].isnull()] #check #13

,LoanNumber,BorrowerState,full_add,Lat,Long,FIPS_z
1320,4357417000,IL,"18960 County Hwy 5 W, EAST DUBUQUE, IL",42.508105,-90.565332,NaN
1968,6227707103,IL,"22069 US HIGHWAY 34, PRINCETON, IL",40.392967,-104.634462,NaN
1992,4428188509,IL,"22069 US Highway 34, Princeton, IL",40.392967,-104.634462,NaN
2078,3984847201,IL,"14 N 845 US-20, Hampshire, IL",44.370091,-108.038415,NaN
3141,2252087100,IL,"14 N 850 Route 20, HAMPSHIRE, IL",42.468637,-73.419886,NaN
3185,3313488304,IL,"43451 N US Highway 41, Zion, IL",35.242488,-85.690479,NaN
3547,6478417409,IL,"1501 W. MAIN ST PO Drawer E, FAIRFIELD, IL",38.033561,-78.497707,NaN
3619,6829277004,IL,"43451 US HIGHWAY 41, ZION, IL",47.046908,-88.518619,NaN
3801,6463967004,IL,"11600 Sterling Parkway, RICHMOND, IL",42.495414,-88.320057,NaN
3832,7639977104,IL,"7505 S Us Highway 66, GARDNER, IL",36.145096,-96.002364,NaN


Parse for Merge

In [13]:
cols= ["LoanNumber", "FIPS_z"]
fips_fill_parsed = fips_fill[cols]
fips_fill_parsed

,LoanNumber,FIPS_z
465,9471527103,170318330001046
1320,4357417000,NaN
1968,6227707103,NaN
1992,4428188509,NaN
2078,3984847201,NaN
3141,2252087100,NaN
3185,3313488304,NaN
3547,6478417409,NaN
3619,6829277004,NaN
3801,6463967004,NaN


Read in base data

In [14]:

#if from FIPS file
fips_check = pd.read_csv("../../../data/state_data/geo/geo_fips/23/IL_fips_scraped.csv") 
#fips_check = fips_check.drop(["Unnamed: 0.1", "Unnamed: 0.1.1", "Unnamed: 0"], axis =1) 
fips_check.head()

,Unnamed: 0,LoanNumber,BorrowerState,full_add,Lat,Long,FIPS_z
0,308139,1060027105,IL,"1751 Lake Cook Road Suite 600, DEERFIELD, IL",42.150930,-87.870512,1.703180e+14
1,308140,1255837210,IL,"One South Wacker Drive, 36th Floor, CHICAGO, IL",41.878166,-87.636672,1.703184e+14
2,308141,1314447103,IL,"870 Oak ST. 0.0, Lake in the Hills, IL",42.189417,-88.309248,1.711187e+14
3,308142,2083627203,IL,"700 Commerce Drive Suite 140, Oak Brook, IL",41.851485,-87.937622,1.704384e+14
4,308143,2142698210,IL,"One E. Oak Hill Drive, Suite 200 0.0, Westmont...",41.815009,-87.951864,1.704384e+14


Merge with MainFrame

In [15]:
fips_merge = fips_check.merge(fips_fill_parsed, on="LoanNumber", how="left")
fips_merge

,Unnamed: 0,LoanNumber,BorrowerState,full_add,Lat,Long,FIPS_z_x,FIPS_z_y
0,308139,1060027105,IL,"1751 Lake Cook Road Suite 600, DEERFIELD, IL",42.150930,-87.870512,1.703180e+14,NaN
1,308140,1255837210,IL,"One South Wacker Drive, 36th Floor, CHICAGO, IL",41.878166,-87.636672,1.703184e+14,NaN
2,308141,1314447103,IL,"870 Oak ST. 0.0, Lake in the Hills, IL",42.189417,-88.309248,1.711187e+14,NaN
3,308142,2083627203,IL,"700 Commerce Drive Suite 140, Oak Brook, IL",41.851485,-87.937622,1.704384e+14,NaN
4,308143,2142698210,IL,"One E. Oak Hill Drive, Suite 200 0.0, Westmont...",41.815009,-87.951864,1.704384e+14,NaN
...,...,...,...,...,...,...,...,...
40758,348897,9752217002,IL,"2531 E. Devon Ave, ELK GROVE VILLAGE, IL",41.993038,-87.941377,1.704384e+14,NaN
40759,348898,9780317408,IL,"9341 Adam Don Pkwy, WOODRIDGE, IL",41.713393,-88.027819,1.704385e+14,NaN
40760,348899,9883677109,IL,"1305 WILEY ROAD STE 125, SCHAUMBURG, IL",42.058747,-88.047553,1.703180e+14,NaN
40761,348900,9887958310,IL,"2095 N Rand Rd, Palatine, IL",42.148603,-88.030347,1.703180e+14,NaN


<b>Fill FIPS Columns - if FIPS file

In [16]:
fips_merge["FIPS_z"] = fips_merge["FIPS_z_y"].fillna(fips_merge["FIPS_z_x"])
fips_merge.head()

,Unnamed: 0,LoanNumber,BorrowerState,full_add,Lat,Long,FIPS_z_x,FIPS_z_y,FIPS_z
0,308139,1060027105,IL,"1751 Lake Cook Road Suite 600, DEERFIELD, IL",42.150930,-87.870512,1.703180e+14,NaN,170318016011002.0
1,308140,1255837210,IL,"One South Wacker Drive, 36th Floor, CHICAGO, IL",41.878166,-87.636672,1.703184e+14,NaN,170318391002007.0
2,308141,1314447103,IL,"870 Oak ST. 0.0, Lake in the Hills, IL",42.189417,-88.309248,1.711187e+14,NaN,171118712051000.0
3,308142,2083627203,IL,"700 Commerce Drive Suite 140, Oak Brook, IL",41.851485,-87.937622,1.704384e+14,NaN,170438446011053.0
4,308143,2142698210,IL,"One E. Oak Hill Drive, Suite 200 0.0, Westmont...",41.815009,-87.951864,1.704384e+14,NaN,170438447022009.0


Drop Extra FIPS Columns

In [17]:
fips_merge_drop = fips_merge.drop(["FIPS_z_x", "FIPS_z_y"], axis=1)
fips_merge_drop = fips_merge_drop.drop(["Unnamed: 0"], axis=1)
fips_merge_drop

,LoanNumber,BorrowerState,full_add,Lat,Long,FIPS_z
0,1060027105,IL,"1751 Lake Cook Road Suite 600, DEERFIELD, IL",42.150930,-87.870512,170318016011002.0
1,1255837210,IL,"One South Wacker Drive, 36th Floor, CHICAGO, IL",41.878166,-87.636672,170318391002007.0
2,1314447103,IL,"870 Oak ST. 0.0, Lake in the Hills, IL",42.189417,-88.309248,171118712051000.0
3,2083627203,IL,"700 Commerce Drive Suite 140, Oak Brook, IL",41.851485,-87.937622,170438446011053.0
4,2142698210,IL,"One E. Oak Hill Drive, Suite 200 0.0, Westmont...",41.815009,-87.951864,170438447022009.0
...,...,...,...,...,...,...
40758,9752217002,IL,"2531 E. Devon Ave, ELK GROVE VILLAGE, IL",41.993038,-87.941377,170438400002003.0
40759,9780317408,IL,"9341 Adam Don Pkwy, WOODRIDGE, IL",41.713393,-88.027819,170438458082006.0
40760,9883677109,IL,"1305 WILEY ROAD STE 125, SCHAUMBURG, IL",42.058747,-88.047553,170318046033000.0
40761,9887958310,IL,"2095 N Rand Rd, Palatine, IL",42.148603,-88.030347,170318036083005.0


In [18]:
fips_merge_drop.loc[fips_merge_drop["FIPS_z"].isnull()] #13

,LoanNumber,BorrowerState,full_add,Lat,Long,FIPS_z
1320,4357417000,IL,"18960 County Hwy 5 W, EAST DUBUQUE, IL",42.508105,-90.565332,NaN
1968,6227707103,IL,"22069 US HIGHWAY 34, PRINCETON, IL",40.392967,-104.634462,NaN
1992,4428188509,IL,"22069 US Highway 34, Princeton, IL",40.392967,-104.634462,NaN
2078,3984847201,IL,"14 N 845 US-20, Hampshire, IL",44.370091,-108.038415,NaN
3141,2252087100,IL,"14 N 850 Route 20, HAMPSHIRE, IL",42.468637,-73.419886,NaN
3185,3313488304,IL,"43451 N US Highway 41, Zion, IL",35.242488,-85.690479,NaN
3547,6478417409,IL,"1501 W. MAIN ST PO Drawer E, FAIRFIELD, IL",38.033561,-78.497707,NaN
3619,6829277004,IL,"43451 US HIGHWAY 41, ZION, IL",47.046908,-88.518619,NaN
3801,6463967004,IL,"11600 Sterling Parkway, RICHMOND, IL",42.495414,-88.320057,NaN
3832,7639977104,IL,"7505 S Us Highway 66, GARDNER, IL",36.145096,-96.002364,NaN


Convert to File

In [19]:
fips_IL = fips_merge_drop

In [20]:
fips_IL.to_csv("../../../data/state_data/geo/geo_fips/23/IL_fips_scraped.csv") 